In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [11]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [12]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item[userid_to_id[user]],   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]]
    return res

In [21]:
def ap_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list,bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k):
        
        if flags[i]:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / k
    
    return result

In [3]:
data = pd.read_csv('D:/КУРСЫ/GeekBrains/Рекомендательные системы/webinar_2/retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(5)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,random_state=42)

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

D:\Users\aavramenko\anaconda4\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 4.78 s


In [10]:
[id_to_itemid[rec] for rec in recs[0]]

[1106523, 1133018, 5569230, 1082185, 1068719]

In [13]:
%%time

model = AlternatingLeastSquares(factors=10, 
                                regularization=0.001,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).tocsr(),   # На вход item-user matrix
          show_progress=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Wall time: 3.56 s


In [14]:
%%time
    
result['als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 2.09 s


0.18863858961801855

In [15]:
%%time

model = AlternatingLeastSquares(factors=10, 
                                regularization=0.001,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).tocsr(),   # На вход item-user matrix
          show_progress=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Wall time: 3.93 s


In [16]:
%%time
    
result['my_als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['my_als'], row['actual']), axis=1).mean()

Wall time: 1.86 s


0.19020568070518803

In [17]:
tfidf_user_item_matrix = tfidf_weight(user_item_matrix).tocsr() # Применяется к item-user матрице ! 

In [18]:
%%time

model = AlternatingLeastSquares(factors=70, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42)

model.fit(tfidf_user_item_matrix,  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

  0%|          | 0/5 [00:00<?, ?it/s]

Wall time: 5.87 s


0.24603330068559914

In [19]:
result

,user_id,actual,als,my_als,als_tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[862349, 1082185, 981760, 995242, 6534178]","[862349, 1082185, 981760, 885290, 995242]","[1082185, 995242, 1100972, 1005186, 856942]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951590, 1029743, 883404, 8090521, 1082185]","[1029743, 8090521, 883404, 951590, 1082185]","[951590, 826249, 1106523, 910032, 1092026]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[834484, 854852, 1037863, 1026118, 845208]","[834484, 1037863, 854852, 845208, 1024306]","[1082185, 1024306, 878996, 1127831, 965267]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 6534178, 995242, 1029743, 981760]","[1082185, 6534178, 1029743, 995242, 981760]","[849843, 1082185, 6534178, 1003188, 938700]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 1029743, 981760, 6534178, 995242]","[1082185, 1029743, 995242, 981760, 6534178]","[1029743, 840361, 995242, 883404, 1082185]"
...,...,...,...,...,...
2037,2496,[6534178],"[844179, 1004906, 1106523, 826249, 899624]","[1004906, 844179, 826249, 1106523, 899624]","[844179, 840361, 1082185, 916122, 995785]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[899624, 844179, 923746, 1004906, 1056509]","[899624, 844179, 833025, 1004906, 923746]","[5569230, 951590, 826249, 845208, 899624]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1082185, 6534178, 1029743, 995242, 981760]","[1082185, 6534178, 1029743, 995242, 981760]","[995242, 862349, 1070820, 6534178, 840361]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1082185, 1029743, 883404, 826249, 1098066]","[1082185, 1029743, 883404, 826249, 1098066]","[883404, 826249, 1029743, 6534178, 1098066]"


In [22]:
result.apply(lambda row: ap_at_k(row['als'], row['actual']), axis=1).mean()

0.12464903689193527

In [23]:
result.apply(lambda row: ap_at_k(row['my_als'], row['actual']), axis=1).mean()

0.12567744041789017

In [24]:
result.apply(lambda row: ap_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.17014365001632384

Лучшая модель по метрикам - als_tfidf